In [1]:
import pandas as pd

In [2]:
import requests

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
df = pd.DataFrame(documents, columns=['course', 'section', 'question', 'text'])
df.head()

,course,section,question,text
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...
1,data-engineering-zoomcamp,General course-related questions,Course - What are the prerequisites for this c...,GitHub - DataTalksClub data-engineering-zoomca...
2,data-engineering-zoomcamp,General course-related questions,Course - Can I still join the course after the...,"Yes, even if you don't register, you're still ..."
3,data-engineering-zoomcamp,General course-related questions,Course - I have registered for the Data Engine...,You don't need it. You're accepted. You can al...
4,data-engineering-zoomcamp,General course-related questions,Course - What can I do before the course starts?,You can start by installing and setting up all...


In [4]:
df[df.course == 'data-engineering-zoomcamp'].head()

,course,section,question,text
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...
1,data-engineering-zoomcamp,General course-related questions,Course - What are the prerequisites for this c...,GitHub - DataTalksClub data-engineering-zoomca...
2,data-engineering-zoomcamp,General course-related questions,Course - Can I still join the course after the...,"Yes, even if you don't register, you're still ..."
3,data-engineering-zoomcamp,General course-related questions,Course - I have registered for the Data Engine...,You don't need it. You're accepted. You can al...
4,data-engineering-zoomcamp,General course-related questions,Course - What can I do before the course starts?,You can start by installing and setting up all...


### Example Documents
Transform example document into a matrix using CountVectorizer()

In [5]:
docs_example = [
    "January course details, register now",
    "Course prerequisites listed in January catalog",
    "Submit January course homework by end of month",
    "Register for January course, no prerequisites",
    "January course setup: Python and Google Cloud"
]

### Vector spaces

- turn the docs into vectors
- term-document matrix:
    - rows: documents
    - columns: words/tokens
- bag of words:
    - word order is lost
    - sparse matrix

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
cv = CountVectorizer()

In [8]:
cv.fit(docs_example)

CountVectorizer()

In [9]:
names = cv.get_feature_names_out()
names

array(['and', 'by', 'catalog', 'cloud', 'course', 'details', 'end', 'for',
       'google', 'homework', 'in', 'january', 'listed', 'month', 'no',
       'now', 'of', 'prerequisites', 'python', 'register', 'setup',
       'submit'], dtype=object)

### Count words through the array of docs
Each 1 represent the existence of said word within sentence in the array docs_example while 0 represent the inexistence of it.

In [10]:
X = cv.transform(docs_example)

In [11]:
X.toarray()

array([[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0],
       [1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]],
      dtype=int64)

In [12]:
df_docs = pd.DataFrame(X.toarray(), columns=names).T
df_docs

,0,1,2,3,4
and,0,0,0,0,1
by,0,0,1,0,0
catalog,0,1,0,0,0
cloud,0,0,0,0,1
course,1,1,1,1,1
details,1,0,0,0,0
end,0,0,1,0,0
for,0,0,0,1,0
google,0,0,0,0,1
homework,0,0,1,0,0


In [13]:
cv = CountVectorizer(stop_words='english')
X = cv.fit_transform(docs_example)

names = cv.get_feature_names_out()

df_docs = pd.DataFrame(X.toarray(), columns=names).T
df_docs

,0,1,2,3,4
catalog,0,1,0,0,0
cloud,0,0,0,0,1
course,1,1,1,1,1
details,1,0,0,0,0
end,0,0,1,0,0
google,0,0,0,0,1
homework,0,0,1,0,0
january,1,1,1,1,1
listed,0,1,0,0,0
month,0,0,1,0,0


### TfidfVectorizer()
For us to categorize the more importance to less frequent terms, we implemented TfidfVectorizer() to process the combining scores and weighting different fields in search.

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
cv = TfidfVectorizer(stop_words='english')
X = cv.fit_transform(docs_example)

names = cv.get_feature_names_out()

df_docs = pd.DataFrame(X.toarray(), columns=names).T
df_docs.round(2)

,0,1,2,3,4
catalog,0.00,0.57,0.00,0.00,0.00
cloud,0.00,0.00,0.00,0.00,0.47
course,0.33,0.27,0.23,0.36,0.23
details,0.69,0.00,0.00,0.00,0.00
end,0.00,0.00,0.47,0.00,0.00
google,0.00,0.00,0.00,0.00,0.47
homework,0.00,0.00,0.47,0.00,0.00
january,0.33,0.27,0.23,0.36,0.23
listed,0.00,0.57,0.00,0.00,0.00
month,0.00,0.00,0.47,0.00,0.00


### Query-Document Similarity
This is how we represent the query in the same vector.

We apply the vectorizer with the query question to turn it into an array-like of document with weight importance for each words.

In [16]:
query = "Do I need to know python to sign up for the January course?"

q = cv.transform([query])
q.toarray()

array([[0.        , 0.        , 0.39515588, 0.        , 0.        ,
        0.        , 0.        , 0.39515588, 0.        , 0.        ,
        0.        , 0.829279  , 0.        , 0.        , 0.        ]])

To see the words of the query and words of some document, we create a dict with each word as key and importance weight of that word as value.

In [17]:
query_dict = dict(zip(names, q.toarray()[0]))
query_dict

doc_dict = dict(zip(names, X.toarray()[0]))
doc_dict

{'catalog': 0.0,
 'cloud': 0.0,
 'course': 0.32842677961242445,
 'details': 0.689240475622327,
 'end': 0.0,
 'google': 0.0,
 'homework': 0.0,
 'january': 0.32842677961242445,
 'listed': 0.0,
 'month': 0.0,
 'prerequisites': 0.0,
 'python': 0.0,
 'register': 0.556074875921159,
 'setup': 0.0,
 'submit': 0.0}

As expected the more words in common, the better the matching score.

In [18]:
df_qd = pd.DataFrame(
    [query_dict, doc_dict],
    index = ['query', 'docs']
).T

print(f"This is the dot-product using manual calculation: {(df_qd['query'] * df_qd['docs']).sum()}")
print(f"This is using matrix multiplication: {X.dot(q.T).toarray()}")

This is the dot-product using manual calculation: 0.25955954945384224
This is using matrix multiplication: [[0.25955955]
 [0.21371415]
 [0.17843726]
 [0.28419115]
 [0.57137158]]


From above, we can also use the cosine similarity to calculate the matrix multiplication as well

In [19]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(X, q)

array([[0.25955955],
       [0.21371415],
       [0.17843726],
       [0.28419115],
       [0.57137158]])

### Apply to all the documents

In [20]:
fields = ['section', 'question', 'text']
transformers = {}
matrices = {}

for field in fields:
    cv = TfidfVectorizer(stop_words='english', min_df=3)
    X = cv.fit_transform(df[field])
    
    transformers[field] = cv
    matrices[field] = X
    
transformers['text'].get_feature_names_out()
matrices['text']

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 26463 stored elements and shape (948, 2118)>

### Search
Try search with the text field by using sample query question.

In [21]:
query = "I just signed up. Is it too late to join the course?"

q = transformers['text'].transform([query])
## Calculate the cosine similarity of query text
score = cosine_similarity(matrices['text'], q).flatten()

In [22]:
mask = (df.course == 'data-engineering-zoomcamp').values
score = score * mask

In [23]:
import numpy as np

idx = np.argsort(-score)[:10]
# idx = np.argpartition(-score, kth=10)

In [24]:
df.iloc[idx].text

0      The purpose of this document is to capture fre...
15     No, late submissions are not allowed. But if t...
22     It's up to you which platform and environment ...
27     You can do most of the course without a cloud....
38     You will have two attempts for a project. If t...
287    This error could result if you are using some ...
3      You don't need it. You're accepted. You can al...
7      Yes, we will keep all the materials after the ...
113    In the join queries, if we mention the column ...
11     No, you can only get a certificate if you fini...
Name: text, dtype: object

### Search with all the fields and boosting + filtering

We can apply it for all the fields. For example, the question field to give it more importance than to others.

In [25]:
boost = {'question':3.0}

score = np.zeros(len(df))

for f in fields:
    b = boost.get(f, 1.0)
    q = transformers[f].transform([query])
    s = cosine_similarity(matrices[f], q).flatten()
    score = score + b * s

Filtering only one type of through course field.

In [26]:
filters = {
    'course':'data-engineering-course'
}

for field, value in filters.items():
    mask = (df[field] == value).values
    score = score * mask

idx = np.argsort(-score)[:10]
results = df.iloc[idx]
results.to_dict(orient='records')

[{'course': 'data-engineering-zoomcamp',
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel."},
 {'course': 'machine-learning-zoomcamp',
  'section': '5. Deploying Machine Learning Models',
  'question': 'Installing waitress on Windows via GitBash: “waitress-serve” command not found',
  'text': 'Running \'pip install waitress\' as a command on GitBash was not downloading the executable file \'waitress-serve.exe\'. You need this file to be able to run commands with wait